<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Keppler-using-geojson" data-toc-modified-id="Keppler-using-geojson-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Keppler using geojson</a></span></li></ul></div>

![portada](https://miro.medium.com/max/7077/1*0l-WeqdtYRruoV_XxS-ZKg.png)

#  Libraries

In [1]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
import rtree
import pygeos


/home/anagarcia/anaconda3/envs/final/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Keppler using geojson

🏬🏬  **Datasets Block** 🏬🏬   ==> https://opendata.vancouver.ca/explore/dataset/block-outlines/information/  

Datasets Graffiti ==> https://opendata.vancouver.ca/explore/dataset/graffiti/export/

To perform the map we use two datasets containing data on the outlines of the blocks, and one containing the location of graffiti in the city.

In [2]:
block = gpd.read_file('Input/block-outlines.geojson')
graffiti = gpd.read_file('Input/graffiti.geojson')


In [3]:
block.head()

,geometry
0,"POLYGON ((-123.03461 49.25223, -123.03488 49.2..."
1,"POLYGON ((-123.04151 49.24497, -123.04162 49.2..."
2,"POLYGON ((-123.03462 49.24779, -123.03476 49.2..."
3,"POLYGON ((-123.03462 49.24868, -123.03476 49.2..."
4,"POLYGON ((-123.03924 49.24666, -123.03924 49.2..."


In [4]:
graffiti.head()

,count,geo_local_area,geometry
0,11,Fairview,POINT (-123.13525 49.26322)
1,10,Downtown,POINT (-123.11055 49.28277)
2,2,Grandview-Woodland,POINT (-123.07426 49.28337)
3,5,Grandview-Woodland,POINT (-123.06075 49.28144)
4,1,West End,POINT (-123.13400 49.28598)


In [5]:
null_block = block.isnull().sum()
null_graf = graffiti.isnull().sum()

print(null_block, null_graf)

geometry    1
dtype: int64 count             0
geo_local_area    1
geometry          0
dtype: int64


In [6]:
block.dropna(inplace=True)
graffiti.dropna(inplace=True)

GeoPandas .sjoin is somewhat similar to a SQL join, but instead of looking at some index, it will look at the geometries — that means, it checks if the points are inside our polygon and return a row for each match.

![sjoin](https://i.stack.imgur.com/aP3px.png)

In [7]:
# join datasets
graf_block = gpd.sjoin(block, graffiti, how='inner', op='contains')

In [8]:
graf_block.reset_index(inplace = True)
graf_block.head()

,index,geometry,index_right,count,geo_local_area
0,0,"POLYGON ((-123.03461 49.25223, -123.03488 49.2...",4111,1,Renfrew-Collingwood
1,0,"POLYGON ((-123.03461 49.25223, -123.03488 49.2...",7136,1,Renfrew-Collingwood
2,9,"POLYGON ((-123.03898 49.24496, -123.03898 49.2...",827,1,Renfrew-Collingwood
3,10,"POLYGON ((-123.04248 49.24147, -123.04235 49.2...",6982,1,Renfrew-Collingwood
4,12,"POLYGON ((-123.04379 49.23646, -123.04402 49.2...",3463,6,Renfrew-Collingwood


In [9]:
#Then we can dissolve our new data frame; this will group the old indexes and sum the graffiti count.graf_block = graf_block.dissolve(by='index', aggfunc='sum')
graf_block = graf_block.dissolve(by='index', aggfunc='sum')
graf_block.head()

,geometry,index_right,count
index,,,
0,"POLYGON ((-123.03640 49.25188, -123.03640 49.2...",11247,2
9,"POLYGON ((-123.03898 49.24496, -123.03898 49.2...",827,1
10,"POLYGON ((-123.04248 49.24147, -123.04235 49.2...",6982,1
12,"POLYGON ((-123.04379 49.23646, -123.04402 49.2...",3463,6
13,"POLYGON ((-123.04310 49.23813, -123.04348 49.2...",6665,4


In [10]:
data_dict = {"graffiti": graf_block, "block": block}
data_dict.keys()

dict_keys(['graffiti', 'block'])

In [11]:
graffiti_map = KeplerGl(height=500, data=data_dict)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [12]:
graffiti_map

KeplerGl(data={'graffiti':                                                 geometry  index_right  count
index …

In [16]:
geojson_config = graffiti_map.config